# Example of using DOpt Federov Exchange Algorithm
## Algorithm obtained from
-  **Algorithm AS 295:** A Fedorov Exchange Algorithm for D-Optimal Design
-  **Author(s):** Alan J. Miller and Nam-Ky Nguyen
-  **Source:** Journal of the Royal Statistical Society. Series C (Applied Statistics), Vol. 43, No. 4, pp. 669-677, 1994
-  **Stable URL:** http://www.jstor.org/stable/2986264

## Source code from
-  http://ftp.uni-bayreuth.de/math/statlib/apstat/

# Load the dopt shared library that provides the interface

### Print the documentation and note that
### Input
-  $x$ is the 2D numpy array that contains the candidate points to select from
-  $n$ is the number of points in the final design
-  $in$ is the number of preselected points that MUST be in the final design (>= 0)
-  $rstart$ indicate if a random start should be performed, should be True in most cases.  If False the user must supply the initial design in $picked$
-  $picked$ is a 1D array that contains the preselected point ID's (remember FORTRAN is 1 based array) on input.  The first $in$ entries are read for ID's.  On output it contains the ID's in x of the final selection

### Output
-  $lndet$ is the logarithm of the determinant of the best design
-  $ifault$ is possible fault codes
>-  -1 if no full rank starting design is found
>-  0 if no error is detected
>-  1* if DIM1 < NCAND
>-  2* if K < N
>-  4* if NRBAR < K(K - 1)/2
>-  8* if K KIN + NBLOCK
>-  16* if the sum of block sizes is not equal to N
>-  32* if any IN(I) < 0 or any IN(I) > BLKSIZ(I)

In [1]:
import numpy as np
import math as m
import dopt
print( dopt.dopt.__doc__ )

lndet,ifault = dopt(x,n,in,rstart,picked)

Wrapper for ``dopt``.

Parameters
----------
x : input rank-2 array('d') with bounds (dim1,kin)
n : input int
in : input int
rstart : input int
picked : input rank-1 array('i') with bounds (n)

Returns
-------
lndet : float
ifault : int



# Define a sample set of data and setup all parameters to call the interface
-  Note that for the picked array we need to define the entry type as np.int32
-  Example problem obtained from: https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/D-Optimal_Designs.pdf
>-  3 Design variables, Full Quadratic model
>-  27 Candidate points in design
>-  Would like to select 10 D-Optimal points

In [2]:
# Sample data set
data = [
    [ -1, -1, -1 ],
    [ 0, -1, -1 ],
    [ 1, -1, -1 ],
    [ -1, 0, -1 ],
    [ 0, 0, -1 ],
    [ 1, 0, -1 ],
    [ -1, 1, -1 ],
    [ 0, 1, -1 ],
    [ 1, 1, -1 ],
    [ -1, -1, 0 ],
    [ 0, -1, 0 ],
    [ 1, -1, 0 ],
    [ -1, 0, 0 ],
    [ 0, 0, 0 ],
    [ 1, 0, 0 ],
    [ -1, 1, 0 ],
    [ 0, 1, 0 ],
    [ 1, 1, 0 ],
    [ -1, -1, 1 ],
    [ 0, -1, 1 ],
    [ 1, -1, 1 ],
    [ -1, 0, 1 ],
    [ 0, 0, 1 ],
    [ 1, 0, 1 ],
    [ -1, 1, 1 ],
    [ 0, 1, 1 ],
    [ 1, 1, 1 ] ]

# Create numpy array to store model matrix
x = np.zeros( (len(data), 10), float, order='F' )

# Create model matrix from data set - remember this is a 3 variable, full quadratic model
for i in range(len(data)):
    
    A = data[i][0]
    B = data[i][1]
    C = data[i][2]
    
    x[i, 0] = 1.0
    
    x[i, 1] = A
    x[i, 2] = B
    x[i, 3] = C
    
    x[i, 4] = A * A
    x[i, 5] = A * B
    x[i, 6] = A * C
    
    x[i, 7] = B * B
    x[i, 8] = B * C
    
    x[i, 9] = C * C
print( x )
print( x.shape )

# Number of points to pick
n = np.int32(10)

# Array of point ID's that will be picked
picked = np.zeros( n, np.int32 , order='F')

# Preselected points that must be in the array
#picked[0] = 3
#picked[1] = 4

# Number of picked points
npicked = np.int32(0)

[[ 1. -1. -1. -1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0. -1. -1.  0.  0.  0.  1.  1.  1.]
 [ 1.  1. -1. -1.  1. -1. -1.  1.  1.  1.]
 [ 1. -1.  0. -1.  1.  0.  1.  0.  0.  1.]
 [ 1.  0.  0. -1.  0.  0.  0.  0.  0.  1.]
 [ 1.  1.  0. -1.  1.  0. -1.  0.  0.  1.]
 [ 1. -1.  1. -1.  1. -1.  1.  1. -1.  1.]
 [ 1.  0.  1. -1.  0.  0.  0.  1. -1.  1.]
 [ 1.  1.  1. -1.  1.  1. -1.  1. -1.  1.]
 [ 1. -1. -1.  0.  1.  1.  0.  1.  0.  0.]
 [ 1.  0. -1.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  1. -1.  0.  1. -1.  0.  1.  0.  0.]
 [ 1. -1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1. -1.  1.  0.  1. -1.  0.  1.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  1.  1.  0.  1.  1.  0.  1.  0.  0.]
 [ 1. -1. -1.  1.  1.  1. -1.  1. -1.  1.]
 [ 1.  0. -1.  1.  0.  0.  0.  1. -1.  1.]
 [ 1.  1. -1.  1.  1. -1.  1.  1. -1.  1.]
 [ 1. -1.  0.  1.  1.  0. -1.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
 [ 1.  1.  

# Call the interface and print the output and the picked array
-  The reported maximum determinant for this design is 1327104
-  We raise an exception with iFault is not 0 - this is just good practice
-  We repeat the DOptimal process 10 times and pick the best design.  We do this in an attempt to avoid local minima

In [3]:
# Store the best design and the corresponding determinant values
bestDes = np.copy( picked )
bestDet = 0.0

# Repeat the process 10 times and store the best design
for i in range(0, 10) :
    picked = np.random.randint( 1, x.shape[0]+1, n, np.int32 )
    lnDet, iFault = dopt.dopt( x, np.int32(n), np.int32(npicked), False, picked )
    print(lnDet)
    
    # Raise an exception if iFault is not equal to 0
    if iFault != 0:
        raise ValueError( "Non-zero return code form dopt algorith.  iFault = ", iFault )
       
    # Store the best design
    if m.fabs(lnDet) > bestDet:
        bestDet =m.fabs(lnDet)
        bestDes = np.copy( picked )

# Print the best design out
print( "Maximum Determinant Found:", bestDet, m.exp(bestDet) )
print( "\nBest Design Found (indices):\n", np.sort(bestDes) )
print( "\nBest Design Found (variables):\n", x[np.sort(bestDes)-1,1:4] )

14.098509682511672
14.098509682511672
14.098509682511674
14.09850968251167
14.098509682511677
13.523145537608112
14.09850968251167
14.098509682511672
13.862943611198904
14.098509682511672
Maximum Determinant Found: 14.098509682511677 1327104.0000000056

Best Design Found (indices):
 [ 1  6  8 11 13 19 21 23 25 27]

Best Design Found (variables):
 [[-1. -1. -1.]
 [ 1.  0. -1.]
 [ 0.  1. -1.]
 [ 0. -1.  0.]
 [-1.  0.  0.]
 [-1. -1.  1.]
 [ 1. -1.  1.]
 [ 0.  0.  1.]
 [-1.  1.  1.]
 [ 1.  1.  1.]]
